In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,George Harrison,A Hard Rain's Gonna Fall,"(george: like to bring on a friend of us all, ..."
1,Nick Cave,Blind Lemon Jefferson,Bline Lemon Jefferson is a-coming. \r\nTap ta...
2,Tom Jones,Dimming Of The Day,This old house is falling down round mice \r\...
3,Xavier Rudd,Things Meant To Be,You can find things meant to be \r\nIf you lo...
4,Nazareth,Broken Down Angel,She used to be someone you could depend on \r...
5,Nicki Minaj,Roman Holiday,"Take your medication, Roman \r\nTake a short ..."
6,Metallica,Trapped Under Ice,I don't know how to live trough this hell \r\...
7,Aerosmith,Let The Music Do The Talking,Rock-a-bye baby if you want to dance \r\nGrab...
8,Megadeth,Sweating Bullets,"Hello me, meet the real me \r\nAnd my misfit'..."
9,Britney Spears,Lonely,"Do you think I'm so naive, \r\nHow dare you p..."


In [10]:
df['text'][0]

"(george: like to bring on a friend of us all, mr. Bob dylan!)  \r\n  \r\nOh, where have you been, my blue-eyed son?  \r\nAnd where have you been, my darling young one?  \r\nI've stumbled on the side of twelve misty mountains,  \r\nI've walked and I've crawled on six crooked highways,  \r\nI've stepped in the middle of seven sad forests,  \r\nI've been out in front of a dozen dead oceans,  \r\nI've been ten thousand miles in the mouth of a graveyard,  \r\nIts a hard, and its a hard, its a hard, its a hard,  \r\nIts a hard rains a-gonna fall.  \r\n  \r\nOh, what did you see, my blue-eyed son?  \r\nAnd what did you see, my darling young one?  \r\nI saw a newborn baby with wild wolves all around it,  \r\nI saw a highway of diamonds with nobody on it,  \r\nI saw a black branch with blood that kept drippin,  \r\nI saw a room full of men with their hammers a-bleedin,  \r\nI saw a white ladder all covered with water,  \r\nI saw ten thousand talkers whose tongues were all broken,  \r\nAnd its 

In [11]:
# df = df.sample(5000)

In [12]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [13]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [14]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [15]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [26]:
similarity[0]

array([1.        , 0.01722706, 0.02374733, ..., 0.02801407, 0.00753239,
       0.09115628])

In [28]:
df[df['song'] == 'Sweating Bullets']

,artist,song,text
8,Megadeth,Sweating Bullets,"hello me , meet the real me and my misfit 's w..."


In [29]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [31]:
recommendation('Sweating Bullets')

['Blood Stained',
 'Anxiety',
 '20 Eyes',
 "It's Gonna Be A Good Day",
 'Hello Love',
 'Just A Little Bit Of Your Heart',
 'Blood On My Hands',
 "We'll Meet Again",
 'Kill The Pain',
 'Get Back In My Life',
 'Mr. Universe',
 'Have A Nice Day',
 'Can You Feel It',
 'Somebody Knows',
 "I'll Miss You",
 'Feels Like Today',
 "Daydreamin'",
 'Little Miss Fit',
 'Public Enemy No 1',
 'Now And Then']

In [32]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))